In [1]:
import os
import librosa
from datetime import datetime
import numpy as np
import pandas as pd
import scipy

In [2]:
PATH = os.getcwd() + '/audio_dataset/'
SAVING_PATH = os.getcwd() + '/MEAN_STFT_npy_dataset/'
print(PATH)
print(SAVING_PATH)

D:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/
D:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/MEAN_STFT_npy_dataset/


In [3]:
NAV_DIR = ["train/Queen/", "train/Queenless/", "test/Queen/", "test/Queenless/", "val/Queen/", "val/Queenless/"]

In [4]:
def read_16k_mono(path):
    y, sr = librosa.load(path, sr=16000, mono=True)
    return y, sr

In [5]:
def stft(y, n_fft, hop_length, win_length, window, center=True):
    feature = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window = window, center=center)
    # print(feature.shape)
    feature_mean = np.mean(np.abs(feature).T,axis=0) 
    # feature_std=np.std(np.abs(feature).T,axis=0)
    # feature_min=np.min(np.abs(feature).T,axis=1)
    # feature_max=np.max(np.abs(feature).T,axis=1)
    # feature_skew = skew(np.abs(feature).T,axis=1)
    # feature_median = np.median(np.abs(feature).T,axis=1)
    # feature = np.hstack((feature_mean,feature_std,feature_min,feature_max,feature_skew,feature_median))
    feature = np.hstack((feature_mean))
    return feature

In [6]:
FILE_PATH = os.listdir(PATH+ NAV_DIR[0])[0]
y, sr = read_16k_mono(PATH + NAV_DIR[0] + FILE_PATH)
feature = stft(y, n_fft=63, hop_length=None, win_length=None, window = "hamming")
feature = feature.reshape(feature.shape[0], 1)
print("STFT feature shape: ", feature.shape)

STFT feature shape:  (32, 1)


**Why axis = 1 but still keep 126 columns??**

In [7]:
def data_extract_stft(path):
    data = []
    print("Extracting data from " + path)
    for file in os.listdir(path):
        if file.endswith(".wav"):
            y, sr = read_16k_mono(path + file)
            feature = stft(y, n_fft=63, hop_length=None, win_length=None, window = "hamming")
            data.append(feature)
    print(f"Extracted {len(data)} files from {path}")
    return data

In [8]:
DATA = []
time0 = datetime.now()
for SUB_PATH in NAV_DIR:
    sub_data = data_extract_stft(PATH + SUB_PATH)
    DATA.append(sub_data)
time1 = datetime.now()
extract_time = (time1 - time0).seconds / 60
print('Extract time: %.2f minutes' % extract_time)

Extracting data from D:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/train/Queen/
Extracted 7000 files from D:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/train/Queen/
Extracting data from D:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/train/Queenless/
Extracted 7000 files from D:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/train/Queenless/
Extracting data from D:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/test/Queen/
Extracted 2000 files from D:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/test/Queen/
Extracting data from D:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/test/Queenless/
Extracted 2000 files from D:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/test/Queenless/
Extracting data from D:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/

In [9]:
print(np.array(DATA[0]).shape)

(7000, 32)


In [10]:
for index in range(len(DATA)):
    np.save(SAVING_PATH + NAV_DIR[index] + NAV_DIR[index].split('/')[-2] + '_STFT.npy', np.array(DATA[index]))
    print('Saved ' + NAV_DIR[index].split('/')[-2] + '_STFT.npy')

Saved Queen_STFT.npy
Saved Queenless_STFT.npy
Saved Queen_STFT.npy
Saved Queenless_STFT.npy
Saved Queen_STFT.npy
Saved Queenless_STFT.npy
